# Part 2.1: Fourier series

In [ ]:
import numpy
from matplotlib import pyplot 
%matplotlib inline
F = 500.0 ; 
Fs = 44100.0;   
T = 1.0/Fs; 
y_a = [0]*500; y_b = [0]*500; y_c = [0]*500;
#summing up sinusoids
for n in range(0,500):
    for m in range(0,500):
        if m%2:
            y_a[n] = y_a[n] + (1.0/m)*numpy.sin(2*numpy.pi*(m*F)*n*T); 
            y_b[n] = y_b[n] + (1.0/(m**2))*numpy.cos(2*numpy.pi*(m*F)*n*T); 
            if m==3:
                y_c[n] = y_c[n] + (1.0/m)*numpy.sin(2*numpy.pi*(m*F)*n*T + numpy.pi/2);
            else:
                y_c[n] = y_c[n] + (1.0/m)*numpy.sin(2*numpy.pi*(m*F)*n*T); 
x = numpy.arange(0,500)*T
fig, ax0 = pyplot.subplots(1)
ax0.plot(x,y_a)   # plot y_a
ax0.set_title("x(k)=Sum(k)((1/k)sin(2*pi*k*F*t)),k=1,3,5...")
ax0.set_xlabel("Time/t") ; ax0.set_ylabel("x(t)")
ax0.grid(True)
fig, ax1 = pyplot.subplots(1)
ax1.plot(x,y_b)   # plot y_b
ax1.set_title("x(k)=Sum(k)((1/(k^2))cos(2*pi*k*F*t)),k=1,3,5...")
ax1.set_xlabel("Time/t") ; ax1.set_ylabel("x(t)")
ax1.grid(True)
fig, ax2 = pyplot.subplots(1)
ax2.plot(x,y_c)   # plot y_c
ax2.set_title("x(k)=Sum(k)((1/k)sin(2*pi*k*F*t)+Q),k=1,3,5...")
ax2.set_xlabel("Time/t") ; ax2.set_ylabel("x(t)")
ax2.grid(True)

Part 2.1 Q&A:

1. Comment on the waveforms obtained for about 4 or more non-zero harmonics.  
    -Waveform of summation of sin waves looks basically like square waves, and that of cos waves looks like triangular waves. The third wavefrom has one of its counterparts shifted left by Π/2， thus the wave looks distorted.
2. What would you expect the waveforms to look like if you could take a very large number of harmonics?  
    -The first wave from will look like square wave, the second resembling triangular wave, the third will still be in strange shape.
3. In what ways are waveforms (a) and (c) similar and different?   
    -They are all summed up by sin waves, but (c) has a harmonic shifted left by Π/2 on k=3 than that of (a).
4. In what ways are waveforms (a) and (b) related to each other?  
    -They are of same amplitude and frequency.
5. Why might waveforms (a) and (c) sound similar over an analog telephone line?  
    -Because for telephone quality speech, ear is considered insensitive to phase, and their frequency components and amplitute of each component are all the same.
6. If the waveform in (a) represented a sequence of pulses sent over an analog telephone line to represent a stream of bits, and the harmonics were affected by phase distortion to produce waveform (c), why might this cause a problem at the receiver?  
    -Since sequence of pulses is sent by digital form, different values are represented by binary values. For wavefom like (a), '1' and '0' can be easily distinguished from each other and error is unlikely to occur when reading it, yet when phase distortion occured, 

# Part 2.2: Frequency-domain processing 

In [ ]:
from scipy.io import wavfile 
import comp28512_utils as utils 
(sampling_freq, y) = wavfile.read("noisySinewave.wav"); #original
print "original noisySinewave"
utils.Audio(y, rate=sampling_freq,)
#print 0-500
fig, ax0 = pyplot.subplots(1)
ax0.set_title("original noisySinewave.wav")
ax0.set_xlabel("Time/t") ; ax0.set_ylabel("Voltage")
ax0.plot(y[0:500])
ax0.grid(True)
Y = numpy.array([0]*500)
y_ext = numpy.array(y[0:500])
#tb deleted
y_ext_full = numpy.array(y)



#FFT
Y = numpy.fft.fft(y_ext)
Y_amp = numpy.sqrt(numpy.power(Y.real,2)+numpy.power(Y.imag,2));
#x with index    #only plot half!
fig, ax0 = pyplot.subplots(1)
ax0.set_title("Amplitude graph of noisySinewave with frequency index")
ax0.set_xlabel("Frequency index k") ; ax0.set_ylabel("Amplitude")
ax0.stem(Y_amp[0:250]/250)
ax0.grid(True)
x_Fs = numpy.arange(500)
x_Fs = x_Fs*sampling_freq/500
#x with Fs       #only plot half!
fig, ax0 = pyplot.subplots(1)
ax0.set_title("Amplitude graph of noisySinewave with frequency")
ax0.set_xlabel("Frequency/Hz") ; ax0.set_ylabel("Amplitude")
ax0.stem(x_Fs[0:250],Y_amp[0:250]/250)
#plot real part
fig, ax0 = pyplot.subplots(1)
ax0.set_title("Real part of noisySinewave, supposed to be symmetric")
ax0.set_xlabel("Frequency index k") ; ax0.set_ylabel("Amplitude in real part")
ax0.stem(Y.real/250)
ax0.grid(True)
#plot imaginary part
fig, ax0 = pyplot.subplots(1)
ax0.set_title("Imaginary part of noisySinewave, supposed to be anti-symmetric")
ax0.set_xlabel("Frequency index k") ; ax0.set_ylabel("Amplitude in real part")
ax0.stem(Y.imag/250)
ax0.grid(True)

#filter
for i in range(0,500):
    if abs(Y_amp[i])<4000*250:
        Y[i]=0+0j
for i in range(1,250):
    if(abs(Y[i])!=0):
        print "remaining freq sample: %dHz"%(i*sampling_freq / 250)
y_ifft = numpy.fft.ifft(Y)
y_ifft = numpy.real(y_ifft)
#use plot to check?
fig, ax0 = pyplot.subplots(1)
ax0.set_title("noisySinewave after noise elimination")
ax0.set_xlabel("Time/t") ; ax0.set_ylabel("Voltage")
ax0.plot(x_Fs,y_ifft)
ax0.grid(True)

Y_full = numpy.fft.fft(y_ext_full)
Y_amp_full = numpy.sqrt(numpy.power(Y_full.real,2)+numpy.power(Y_full.imag,2));
for i in range(0,3*sampling_freq):
    if abs(Y_amp_full[i])<4000*len(y)/2:
        Y_full[i]=0+0j
y_ifft_full = numpy.fft.ifft(Y_full)
y_ifft_full = numpy.real(y_ifft_full)
y_ifft_full = numpy.int16(y_ifft_full)
print "noisySinewave after noise elimination"
utils.Audio(y_ifft_full, rate=sampling_freq,)


Part 2.2 Q&A:

1. If the original time-domain signal has N samples, how many frequency-domain samples do we obtain after performing the FFT?  Why do we only need to plot the first N/2 samples of the magnitude spectrum?  
    -N samples. -Because according to Sampling Theorem, samples larger than N/2 represent frequency components that are filtered out.
2. How is periodic part of the sound and the noise seen in the magnitude spectrum?  
    -Frequency points for music have considerably higher amplitude and are disparsed; those for noise are lower in amplitude and distributed across range of frequency.
3. How do you convert FFT frequency sample ('bin') number to frequency?  
    -The lowest one is fundamental frequency which is Fs/N, frequency of each sample is its number times fundamental frequency, yet samples of frequencies larger than Fs/2 are symmetric with those below.
4. Would you describe the noise as being 'white'?  Please explain your answer.  
    -Yes, because the noise is composed by sound of almost all different frequencies from 0 to Fs(sampling frequency).
5. What value of threshold was chosen and why?  
    -Threshold value is 3000 as it is higher than highest amplitude of noise and lower than lowest amplitude of music.
6. As you must apply the frequency-domain processing to the real and imaginary parts of the FFT individually, why would it be wrong to calculate thresholds for these parts individually rather than for the magnitude spectrum?  
    -Because those parts themselves can't represent amplitude correctly
7. After applying the inverse FFT, is the resulting processed signal real, i.e. does it have zero imaginary part?  If it were not real, what would you conclude from this?  
    -Not real. Even though input of ifft is symmetric, imaginary part of ifft output still exist due to rounding error of CPU.
8. Has any of the noise been removed by this ‘spectral subtraction’ process?  
    -Yes.
9. What Fourier series components are present in the periodic part of the sound?  
    -That of 640Hz & 1280Hz.
10. Did you have any chance of answering question 9 by just observing the time-domain waveform (i.e. if you had never heard of the FFT)?  
    -No, because we cannot distinguish periodic part from original waveform.

# Part 2.3: Transforming music files to & from frequency-domain 

In [ ]:
from scipy import fftpack
x = numpy.arange(1024)
Fs = 44100.0
T = 1.0/Fs
x = x * (1.0/Fs)
y = [0]*1024
for n in range(0,1024):
    y[n] = numpy.sin( 2 * numpy.pi * 441 * n * T )
y_ori = numpy.array(y)
# plot original wave
fig, ax0 = pyplot.subplots(1)
ax0.set_title("Original sine wave at F=441Hz")
ax0.set_xlabel("Time/t") ; ax0.set_ylabel("Voltage/v")
ax0.plot(x,y_ori)
ax0.grid(True)
# DCT
Y = fftpack.dct(y_ori, norm='ortho')
#plot magnitude
fig, ax0 = pyplot.subplots(1)
ax0.set_title("Sine wave at F=441Hz in frequency domain")
ax0.set_xlabel("Freqency index") ; ax0.set_ylabel("Amplitude")
ax0.stem(Y)
ax0.grid(True)
# iDCT
y_rev = fftpack.idct(Y, norm='ortho')
#plot retrieved wave
fig, ax0 = pyplot.subplots(1)
ax0.set_title("Sine wave after idct at F=441Hz")
ax0.set_xlabel("Time/t") ; ax0.set_ylabel("Voltage/v")
ax0.plot(x,y_rev)
ax0.grid(True)

(Fs_msc, wy) = wavfile.read("HQ-music44100-mono.wav")
print "before idct"
utils.Audio(wy, rate=Fs_msc,)
j=0; wdctF=[]; ry=[]
#dct process
while j*1024<len(wy):
    #truncate by length of 1024
    if (j+1)*1024<=len(wy):
        Y_trunc = wy[(j*1024):((j+1)*1024)]
    else:
        Y_trunc = wy[(j*1024):len(wy)]
    dctF = fftpack.dct(Y_trunc, norm='ortho')    #dct
    #record 
    wdctF.extend(dctF)
    j+=1
wdctF = numpy.array(wdctF)
SM=max(abs(wdctF))         # Get maximum amplitude 
wdctF = wdctF/float(SM)    # Scale maximum to 1
wdctF = numpy.array(wdctF * (2**15 - 0.5) -0.5) # expansion to 16-bit
#save file
f = open ("freq.bin","wb")
numpy.save(f,wdctF)
f.close()
#load file
f = open ("freq.bin","rb")
rdctF = numpy.load(f)
#idct process
j=0
while j*1024<len(rdctF):
    #truncate by length of 1024
    if (j+1)*1024<=len(rdctF):
        Y_trunc = rdctF[(j*1024):((j+1)*1024)]
    else:
        Y_trunc = rdctF[(j*1024):(len(rdctF))]
    idctT = fftpack.idct(Y_trunc, norm='ortho')    #dct
    #print len(idctT)
    #record 
    ry.extend(numpy.int16(idctT))
    j+=1
ry = numpy.array(ry)
SM=max(abs(ry))         # Get maximum amplitude 
ry = ry/float(SM)    # Scale maximum to 1
ry = numpy.int16(numpy.array(ry * (2**15 - 0.5) -0.5)) # expansion to 16-bit
#play audio
print "after idct"
utils.Audio(ry, rate=Fs_msc,)
#plot audio file after idct
fig, ax0 = pyplot.subplots(1)
ax0.set_title("Audio file of frequency domain sample")
ax0.set_xlabel("Index") ; ax0.set_ylabel("Amplitude")
ax0.plot(ry)
ax0.grid(True)

Part 2.3 Q&A:
    
1. Why does the music have to be split up into sections when we wish to to apply frequency domain processing?  
    -Because they are not purely periodic, but if we split it into small segments then they'll all be short term periodic, thus can get DCT applied.
2. Does the transformation, saving to a file, reading back from the file and/or reconstruction introduce any noticeable distortion?  
    -No, because only minor changes on data will occur due to rounding error during transformation and reconstruction, save/load do not change data.
3. If there was some distortion, what would be the cause of it?  
    -The cause may be lose of precision due to rounding to int16. 

# Part 2.4: Principles of mp3 encoding

In [ ]:
(sampling_freq, y) = wavfile.read("HQ-music44100-mono.wav")
print "original HQ-music44100-mono.wav"
utils.Audio(y, rate=sampling_freq,)
y_rev = [0]*len(y)
for m in [3,4,5,6,8]: #NB
    for n in [2,6,30,80]:
        wdctF = []
        j = 0
        valve = 100*n
        total_add = 0; total_pre = 0; total_fin = 0
        while j*1024<len(y):
            #truncate by length of 1024
            if (j+1)*1024<=len(y):
                Y_trunc = y[(j*1024):((j+1)*1024)]
            else:
                Y_trunc = y[(j*1024):(len(y))]
            Y = fftpack.dct(Y_trunc, norm='ortho')    #dct
            #delete frequency samples that are too low to hear
            Y[0]=0; Y[1]=0; Y[2]=0
            #apply valve
            for i in range(0,len(Y)):
                if sampling_freq*i/2.0/len(Y)>16000:
                    Y[i] = 0
                    total_pre += 1
            #apply valve
            for i in range(0,len(Y)):
                if abs(Y[i]) < valve: 
                    Y[i] = 0
                    total_add +=1
            total_fin += numpy.count_nonzero(Y)
            #record 
            wdctF.extend(Y)
            j+=1
        #reduce NB
        wdctF = numpy.array(wdctF)
        SM=max(abs(wdctF))         # Get maximum amplitude 
        wdctF = wdctF/float(SM)    # Scale maximum to 1
        wdctF = numpy.round(wdctF * (2**(2*m-1))) # expansion to NB=m
        wdctF = numpy.round(wdctF * (2**(16-2*m))) #restore to original scale
        j=0
        while j*1024<len(wdctF):
            #truncate by length of 1024
            if (j+1)*1024<=len(wdctF):
                Y = wdctF[(j*1024):((j+1)*1024)]
            else:
                Y = wdctF[(j*1024):(len(wdctF))]
            #count reamining nunbers
            ybuff = fftpack.idct(Y, norm='ortho')
            if (j+1)*1024<=len(y):
                y_rev[(j*1024):((j+1)*1024)] = ybuff
            else:
                y_rev[(j*1024):len(y)] = ybuff
            j+=1
        print "NB=%d, filter:A=%d, %d samples eventually, %d samples saved\
        "%(2*m, valve, total_fin, total_add)
        y_rev = numpy.array(y_rev)
        #play y_rev
        SM=max(abs(y_rev))         # Get maximum amplitude 
        y_rev = y_rev/float(SM)    # Scale maximum to 1
        y_rev = numpy.round(y_rev * (2**15 - 1)) # expansion to NB=16
        utils.Audio(numpy.int16(y_rev), rate=sampling_freq,)
        #plot original discontinuity
        if valve==600 and m==4:
            fig, ax0 = pyplot.subplots(1)
            ax0.set_title("Example of discontinuity after frequency domain pocessing, between 3rd and 4th frame")
            ax0.set_xlabel("Time/t") ; ax0.set_ylabel("Voltage")
            ax0.plot(y_rev[3062:3082])
            ax0.grid(True)
print "bit saving after reducing the bandwidth to 16 kHz is %.6d"%(total_pre)
fig, ax0 = pyplot.subplots(1)
ax0.set_title("Frequency domain sample form ")
ax0.set_xlabel("Index k") ; ax0.set_ylabel("Voltage")
ax0.plot(wdctF)
ax0.grid(True)

Part 2.4 Q&A:
    
1. What saving in bit-rate can achieved by reducing the bandwidth to 16 kHz?  
    -241172/802001 = 27.34%.   241172x16/20 = 192,938 bit/s
2. What is the effect of varying the constant threshold?  What happens with a value that (a) is definitely too high, (b) too low and (c) potentially satisfactory?  Give the values.  
    -As threshold become higher, sounds get more blurred since more samples have been filtered out, and clicks become clearer. When threshold=8000, clicks are rather noticable, audio starts to become distorted. When threshold=200, audio sounds basically like original one, yet bit-rate saving is hardly achieved. When threshold=600, audio still sounds like original one, and bit-rate saving is considerable, thus come to a balance.
3. For the 'potentially satisfactory' threshold, what is the effect of quantising the remaining non-zero DCT coefficients to 8 bits per sample?  
    -When NB=8, audio sounds like original version, yet clicking sound becomes noticable.
4. For the 'potentially satisfactory' threshold, and 8 bits per DCT coefficient, assuming that you could send the zero-valued DCT coefficients at negligible cost, what bit-rate is required by this coding procedure?  What bit-rate saving (in percentage terms) is achieved?  
    -bit-rate: 122202*8/20 = 48880 bit/s; bit-rate saving =  (882001×16-122202×8)/(882001×16)=93.07%. 
5. Did you observe any distortion in the form of 'clicks' due to discontinuities at frame boundaries?  
    -Yes.
6. If so, what causes these clicks?  Think carefully about this and consider why you do not get clicks until you start doing some frequency domain processing.  
    -Discintunities between last sample of former frame and first sample of latter frame. Because we are changing values of frequency domain samples, when we convert these frames back to time domain, continuity between frames will be broken and clicks appear .

# Part 2.5: Simple method for eliminating discontinuities 

In [ ]:
import math
print len(y)
x_ind = range(3062,3082)
y_rev = [0]*len(y)
for n in range(1,6):
        wdctF = []
        j=0; z=0
        valve = 600
        while j*1024<len(y):
            #truncate by length of 1024
            if (j+1)*1024<=len(y):
                Y_trunc = y[(j*1024):((j+1)*1024)]
            else:
                Y_trunc = y[(j*1024):(len(y))]
            Y = fftpack.dct(Y_trunc, norm='ortho')    #dct
            #apply valve
            for i in range(0,len(Y)):
                if sampling_freq*i/2.0/len(Y)>16000: 
                    Y[i] = 0
            total_pre += numpy.count_nonzero(Y)
            #apply valve
            for i in range(0,len(Y)):
                if abs(Y[i]) < valve: 
                    Y[i] = 0
            #record 
            wdctF.extend(Y)
            j+=1
        #reduce NB
        wdctF = numpy.array(wdctF)
        SM=max(abs(wdctF))         # Get maximum amplitude 
        wdctF = wdctF/float(SM)    # Scale maximum to 1
        wdctF = numpy.round(wdctF * (2**(8-1) - 1)) # expansion to NB=8
        wdctF = numpy.round(wdctF * (2**(16-8) - 1)) #restore to original scale
        j=0
        while j*1024<len(wdctF):
            #truncate by length of 1024
            if (j+1)*1024<=len(wdctF):
                Y = wdctF[(j*1024):((j+1)*1024)]
            else:
                Y = wdctF[(j*1024):(len(wdctF))]
            ybuff = fftpack.idct(Y, norm='ortho')
            if j>0:  #smoothing by z from last frame
                for k in range(0,2*n):
                    ybuff[k] = (math.e**k*ybuff[k]+z)/(math.e**k+1)
            z = ybuff[len(ybuff)-1] #update z for next frame
            if (j+1)*1024<=len(y):
                y_rev[(j*1024):((j+1)*1024)] = ybuff
            else:
                y_rev[(j*1024):len(y)] = ybuff
            j+=1
        print "use first %d samples for smoothing by z from last frame"%(2*n)
        y_rev = numpy.array(y_rev)
        SM=max(abs(y_rev))         # Get maximum amplitude 
        y_rev = y_rev/float(SM)    # Scale maximum to 1
        y_rev = numpy.round(y_rev * (2**15 - 1)) # expansion to NB=16
        utils.Audio(numpy.int16(y_rev), rate=sampling_freq,)
        fig, ax0 = pyplot.subplots(1)
        ax0.set_title("Smoothing effect of the same part as 2.4 using %d samples"%(2*n))
        ax0.set_xlabel("Index k") ; ax0.set_ylabel("Voltage")
        ax0.plot(x_ind,y_rev[3062:3082])
        ax0.grid(True)

Part 2.5 Q&A:
    
1. Describe the principle of your simple method and how it was implemented.  
    -For each frame I store the last sample, and for first several samples, say x[k], I apply x[k] = (e^k×x[k]+z)/(e^k+1), giving z higher weight in altered sample to get them as close value of z as possible. 
2. Does it work?  If so how well does it work?  
    -It works, though clicks are still noticable, they become slighter.

# Part 2.6: Run-length & Huffman coding

In [ ]:
wdctF = []
y_rev = [0]*len(y)
j=0; z=0
valve = 600
all_count=0; sam_count=0
while j*1024<len(y):
    #truncate by length of 1024
    if (j+1)*1024<=len(y):
        Y_trunc = y[(j*1024):((j+1)*1024)]
    else:
        Y_trunc = y[(j*1024):(len(y))]
    Y = fftpack.dct(Y_trunc, norm='ortho')    #dct
    Y[0]=0; Y[1]=0; Y[2]=0
    if j==0:
        sam_count += 3
    all_count+=3
    #apply valve
    for i in range(0,len(Y)):
        if abs(Y[i]) < valve or sampling_freq*i/2.0/len(Y)>16000: 
            Y[i] = 0  
        else:
            if j==0:
                sam_count += 1
            all_count+=1
    count = 0; flag = 0
    for i in range(0,len(Y)):
        if Y[i]==0 and (i==0 or Y[i-1] != 0):
            count=1
        elif Y[i]==0 and Y[i-1] == 0:
            count+=1
            if (count+1) // 257:
                all_count += 1
                count = 0
                if j==0:
                    sam_count += 1
        elif Y[i]!=0:
            count = 0
    #record 
    wdctF.extend(Y)
    j+=1
#reduce NB
wdctF = numpy.array(wdctF)
SM=max(abs(wdctF))         # Get maximum amplitude 
wdctF = wdctF/float(SM)    # Scale maximum to 1
wdctF = numpy.round(wdctF * (2**(8-1) - 1)) # expansion to NB=8
wdctF = numpy.round(wdctF * (2**(16-8) - 1)) #restore to original scale
j=0
while j*1024<len(rdctF):
    #truncate by length of 1024
    if (j+1)*1024<=len(rdctF):
        Y = wdctF[(j*1024):((j+1)*1024)]
    else:
        Y = wdctF[(j*1024):(len(wdctF))]
    
    ybuff = fftpack.idct(Y, norm='ortho')
    if (j+1)*1024<=len(y):
        y_rev[(j*1024):((j+1)*1024)] = ybuff
    else:
        y_rev[(j*1024):len(y)] = ybuff
    j+=1
#print all_count
sam_save = (1024*8-sam_count*16) / 0.023
sam_save_per = sam_save*0.023/(1024*8)
all_save = (len(y)*8-all_count*16) /20.0
all_save_per = all_save*20.0/(len(y)*8)
print "bit-saving for a typical frame is %d, %.6s, for whole file is %d, %.6s\
"%(sam_save, sam_save_per, all_save,all_save_per )

Part 2.6 Q&A:

1. What bit-rate saving was achieved for the single frame by run-length coding of zeros?   
    -It's 252521, percentage is 70.89%.
2. What bit-rate saving was achieved for the whole file by run-length coding of zeros?  
    -It's 207985, percentage is 58.95%.

# Part 2.7: Eliminating discontinuities using overlapping frames

In [ ]:
for m in [7,8,10,12]: #NB
    for n in [0.8,8,9,30,80]:
        j = 0
        wdctF = []
        valve = 100*n
        total_fin = 0; total_pre = 0
        while j*1024<len(y):
            #truncate by length of 1024
            if (j+1)*1024<=len(y):
                Y_trunc = y[(j*1024):((j+2)*1024)]
            else:
                Y_trunc = y[(j*1024):(len(y))]
            Y_trunc = Y_trunc * numpy.hanning(len(Y_trunc)) #apply hamming window
            Y = fftpack.dct(Y_trunc, norm='ortho')    #dct
            #delete frequency samples that are too low to hear
            Y[0]=0; Y[1]=0; Y[2]=0
            #apply valve
            for i in range(0,len(Y)):
                if sampling_freq*i/2.0/len(Y)>16000: 
                    Y[i] = 0
            total_pre += numpy.count_nonzero(Y)
            #apply valve
            for i in range(0,len(Y)):
                if abs(Y[i]) < valve: 
                    Y[i] = 0
            #record
            wdctF.extend(Y)
            j+=1
        #reduce NB
        wdctF = numpy.array(wdctF)
        SM=max(abs(wdctF))         # Get maximum amplitude 
        wdctF = wdctF/float(SM)    # Scale maximum to 1
        wdctF = numpy.round(wdctF * (2**m - 1)) # expansion to NB=m
        wdctF = numpy.round(wdctF * (2**(16-m) - 1)) #restore to original scale
        y_res=numpy.array([])
        y_pre = numpy.array([0]*1024)
        j=0
        while j*2048<len(wdctF):
            #truncate by length of 1024
            if (j+1)*2048<=len(wdctF):
                Y = wdctF[(j*2048):((j+1)*2048)]
            else:
                Y = wdctF[(j*2048):(len(wdctF))]
            #count reamining numbers
            total_fin += numpy.count_nonzero(Y)
            y_rev = fftpack.idct(Y, norm='ortho')   #iDCT
            #split idct res into 2 parts, 
            #sum the first half of the current received frame with the second half of the previous received frame
            y_rev_pre = numpy.array(y_rev[0:1024]); y_rev_aft = numpy.array(y_rev[1024:len(y_rev)])
            y_res = numpy.append(y_res, y_pre+y_rev_pre) #save to final res
            y_pre = y_rev_aft #prepare for adding of next frame
            j+=1
        #the final last half do not need to add up with other frames, yet it need to be appended to final result
        print "NB=%d, A=%d, remain %d samples, sample per frame: %s\
        "%(m, valve, total_fin, total_fin*1.0/(j-1))
        y_res = numpy.append(y_res, y_pre)
        SM=max(abs(y_res))         # Get maximum amplitude 
        y_res = y_res/float(SM)    # Scale maximum to 1
        y_res = numpy.round(y_res * (2**15 - 1)) # expansion to NB=m
        wavfile.write("NB=%d_A=%d"%(m, valve), sampling_freq, numpy.int16(y_res));
        utils.audio_from_file("NB=%d_A=%d"%(m, valve))  # play rendered speech from file

Part 2.7 Q&A:

1. With the best threshold for constant masking, how many non-zero DCT coefficients are there per frame on average?  
    -A=800, there are 257.07 samples on average.
2. How many bits per DCT coefficient did you find are really needed?  
    -8 bits.
3. If we could find a way of sending the zero valued DCT coefficients at no cost (or very little cost), estimate the bit-rate saving that would result from the three techniques considered above.  
    -bit-rate saving: (1764002×16-220182×8)/20 = 1323128 bit/s   
    -percentage: (1764002×16-220182×8)/(1764002×16)=93.76%.